In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_forecasting import TimeSeriesDataSet, DeepAR, Baseline
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import QuantileLoss, RMSE, MAE, MAPE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Set seed for reproducibility
pl.seed_everything(42)
# Configure plotting
plt.style.use('ggplot')
sns.set_context("notebook", font_scale=1.2)

## 2. Loading and Preparing Data

We'll load our EURUSD data and macroeconomic features, then prepare them for time series forecasting.

In [ ]:
# Load feature data
features = pd.read_csv("../src/features/macro_features.csv", parse_dates=["Date"])
features.set_index("Date", inplace=True)

# Load price data
price_data = pd.read_csv("../src/data_fetch/EURUSD.csv")
price_data['Date'] = pd.to_datetime(price_data['Date'])
price_data.set_index("Date", inplace=True)

# Extract price and create labels for up/down movements
price = price_data["EURUSD_Close"]
price_returns = price.pct_change().dropna()
labels = price_returns.apply(lambda x: 1 if x > 0 else 0)

# Merge data
combined_data = features.join(price.rename("EURUSD_Close"), how="inner")
combined_data = combined_data.join(labels.rename("Label"), how="inner")
combined_data.reset_index(inplace=True)

# Display the first few rows
combined_data.head()

### 2.1 Visualize the target variable

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(combined_data['Date'], combined_data['EURUSD_Close'])
plt.title('EURUSD Exchange Rate')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.tight_layout()
plt.show()

## 3. Data Normalization

Neural networks work best with normalized data. We'll normalize both features and target variables.

In [ ]:
# Define feature columns
feature_columns = [col for col in combined_data.columns 
                   if col not in ['Date', 'Label', 'EURUSD_Close']]

# Initialize scalers
feature_scaler = StandardScaler()
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Split into train and test sets (80% train, 20% test)
train_size = int(len(combined_data) * 0.8)
train_data = combined_data.iloc[:train_size].copy()
test_data = combined_data.iloc[train_size:].copy()

# Fit and transform feature scaler on training data
train_features = train_data[feature_columns]
normalized_train_features = feature_scaler.fit_transform(train_features)

# Transform test features
test_features = test_data[feature_columns]
normalized_test_features = feature_scaler.transform(test_features)

# Update dataframes with normalized features
for i, col in enumerate(feature_columns):
    train_data[col] = normalized_train_features[:, i]
    test_data[col] = normalized_test_features[:, i]

# Normalize target (EURUSD_Close)
train_target = train_data['EURUSD_Close'].values.reshape(-1, 1)
train_data['EURUSD_Close_normalized'] = target_scaler.fit_transform(train_target).flatten()

test_target = test_data['EURUSD_Close'].values.reshape(-1, 1)
test_data['EURUSD_Close_normalized'] = target_scaler.transform(test_target).flatten()

# Display normalized data
train_data[['Date', 'EURUSD_Close', 'EURUSD_Close_normalized'] + feature_columns[:3]].head()

## 4. Preparing TimeSeriesDataSet Format

PyTorch Forecasting requires data in a specific format. We need to prepare our data accordingly.

In [ ]:
# Add required columns for TimeSeriesDataSet
train_data['time_idx'] = range(len(train_data))  # Time index starting from 0
train_data['group_id'] = 'forex'  # All rows belong to the same group

test_data['time_idx'] = range(len(train_data), len(train_data) + len(test_data))
test_data['group_id'] = 'forex'

# Define forecasting parameters
max_encoder_length = 30  # Use 30 days of history
prediction_length = 1    # Predict 1 day ahead

# Create training dataset
training_data = TimeSeriesDataSet(
    data=train_data,
    time_idx="time_idx",
    target="EURUSD_Close_normalized",
    group_ids=["group_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=feature_columns,  # Our macro features are known
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["EURUSD_Close_normalized"],  # Target is unknown in future
    target_normalizer=None,  # We manually normalized the target
    add_relative_time_idx=True,
    add_target_scales=False,
    randomize_length=None,
)

# Create validation dataset
validation_data = TimeSeriesDataSet.from_dataset(
    training_data, 
    test_data, 
    predict=True, 
    stop_randomization=True
)

# Create data loaders
batch_size = 32
train_dataloader = training_data.to_dataloader(
    batch_size=batch_size, 
    num_workers=0, 
    shuffle=True
)
val_dataloader = validation_data.to_dataloader(
    batch_size=batch_size, 
    num_workers=0, 
    shuffle=False
)

## 5. Building and Training the DeepAR Model

Now we'll create and train the DeepAR model.

In [ ]:
# Define the DeepAR model
model = DeepAR.from_dataset(
    training_data,
    learning_rate=0.01,
    hidden_size=64,
    rnn_layers=2,
    dropout=0.3,
    loss=QuantileLoss(),
)

# Configure trainer with early stopping
early_stop_callback = pl.callbacks.EarlyStopping(
    monitor="val_loss", 
    min_delta=1e-4, 
    patience=10, 
    verbose=True, 
    mode="min"
)

trainer = pl.Trainer(
    max_epochs=50,
    accelerator='auto',  # Use GPU if available
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback],
)

# Train the model
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

## 6. Making Predictions

Let's use our trained model to make predictions and visualize the results.

In [ ]:
# Make predictions on validation data
predictions = model.predict(val_dataloader)

# Convert predictions back to original scale
pred_median = predictions.prediction[:, -1, 1].cpu().numpy()  # Median (0.5 quantile)
pred_lower = predictions.prediction[:, -1, 0].cpu().numpy()   # Lower (0.1 quantile)
pred_upper = predictions.prediction[:, -1, 2].cpu().numpy()   # Upper (0.9 quantile)

# Inverse transform to original scale
pred_median_orig = target_scaler.inverse_transform(pred_median.reshape(-1, 1)).flatten()
pred_lower_orig = target_scaler.inverse_transform(pred_lower.reshape(-1, 1)).flatten()
pred_upper_orig = target_scaler.inverse_transform(pred_upper.reshape(-1, 1)).flatten()

# Get actual values
actual_values = test_data['EURUSD_Close'].values

# Create a DataFrame to store results
results_df = pd.DataFrame({
    'Date': test_data['Date'],
    'Actual': actual_values,
    'Predicted': pred_median_orig,
    'Lower_CI': pred_lower_orig,
    'Upper_CI': pred_upper_orig
})

# Show the first few predictions
results_df.head()

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(14, 7))
plt.plot(results_df['Date'], results_df['Actual'], label='Actual', color='blue')
plt.plot(results_df['Date'], results_df['Predicted'], label='Predicted', color='red', linestyle='--')
plt.fill_between(results_df['Date'], results_df['Lower_CI'], results_df['Upper_CI'], 
                 color='red', alpha=0.2, label='90% Prediction Interval')
plt.title('DeepAR Predictions vs Actual EURUSD Exchange Rate')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 7. Generating Trading Signals

Now let's generate trading signals based on our predictions.

In [ ]:
# Generate trading signals based on predicted direction
# 1 for predicted price increase, 0 for predicted decrease
results_df['Signal'] = (results_df['Predicted'] > results_df['Actual'].shift(1)).astype(int)

# Calculate the confidence based on prediction interval width
results_df['Interval_Width'] = results_df['Upper_CI'] - results_df['Lower_CI']
results_df['Confidence'] = 1 - (results_df['Interval_Width'] / results_df['Actual'])

# Calculate actual returns
results_df['Actual_Return'] = results_df['Actual'].pct_change()

# Calculate strategy returns
# For signal=1 (buy), we get the next day's return
# For signal=0 (sell), we get the negative of the next day's return
results_df['Strategy_Return'] = np.where(
    results_df['Signal'] == 1,
    results_df['Actual_Return'].shift(-1),  # Next day's return for buy signals
    -results_df['Actual_Return'].shift(-1)  # Negative of next day's return for sell signals
)

# Calculate cumulative returns
results_df['Cumulative_Market_Return'] = (1 + results_df['Actual_Return']).cumprod() - 1
results_df['Cumulative_Strategy_Return'] = (1 + results_df['Strategy_Return'].fillna(0)).cumprod() - 1

# Remove NaN values
results_df = results_df.dropna()

# Display the results
results_df[['Date', 'Actual', 'Predicted', 'Signal', 'Confidence', 
            'Actual_Return', 'Strategy_Return']].head(10)

In [ ]:
# Plot cumulative returns comparison
plt.figure(figsize=(14, 7))
plt.plot(results_df['Date'], results_df['Cumulative_Market_Return'], 
         label='Market Returns (Buy & Hold)', color='blue')
plt.plot(results_df['Date'], results_df['Cumulative_Strategy_Return'], 
         label='DeepAR Strategy Returns', color='green')
plt.title('Cumulative Returns Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Performance Evaluation

Let's evaluate the performance of our trading strategy.

In [ ]:
# Calculate performance metrics
total_return = results_df['Cumulative_Strategy_Return'].iloc[-1]
annual_return = (1 + total_return) ** (252 / len(results_df)) - 1  # Annualized return assuming 252 trading days
sharpe_ratio = results_df['Strategy_Return'].mean() / results_df['Strategy_Return'].std() * np.sqrt(252)
win_rate = len(results_df[results_df['Strategy_Return'] > 0]) / len(results_df)
max_drawdown = (results_df['Cumulative_Strategy_Return'] / 
                results_df['Cumulative_Strategy_Return'].cummax() - 1).min()

# Print performance summary
print(f"DeepAR Strategy Performance Summary:")
print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annual_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Win Rate: {win_rate:.2%}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")

## 9. Adjusting the Strategy for Different Market Conditions

We can adjust our strategy based on prediction confidence and market volatility.

In [ ]:
# Calculate rolling volatility over 30 days
results_df['Rolling_Volatility'] = results_df['Actual_Return'].rolling(30).std() * np.sqrt(252)  # Annualized

# Create a more conservative strategy that only trades when confidence is high
confidence_threshold = results_df['Confidence'].median()  # Use median confidence as threshold
results_df['Conservative_Signal'] = np.where(
    results_df['Confidence'] > confidence_threshold,
    results_df['Signal'],  # Use original signal if confidence is high
    np.nan  # No trade if confidence is low
)

# Calculate conservative strategy returns
results_df['Conservative_Return'] = np.where(
    results_df['Conservative_Signal'] == 1,
    results_df['Actual_Return'].shift(-1),  # Next day's return for buy signals
    np.where(
        results_df['Conservative_Signal'] == 0,
        -results_df['Actual_Return'].shift(-1),  # Negative of next day's return for sell signals
        0  # No return when not trading
    )
)

# Calculate cumulative returns for conservative strategy
results_df['Cumulative_Conservative_Return'] = (1 + results_df['Conservative_Return'].fillna(0)).cumprod() - 1

# Calculate number of trades for each strategy
original_trades = results_df['Signal'].diff().abs().sum()
conservative_trades = results_df['Conservative_Signal'].dropna().diff().abs().sum()

print(f"Original Strategy Trades: {original_trades}")
print(f"Conservative Strategy Trades: {conservative_trades}")

In [ ]:
# Plot all strategies together
plt.figure(figsize=(14, 7))
plt.plot(results_df['Date'], results_df['Cumulative_Market_Return'], 
         label='Market (Buy & Hold)', color='blue')
plt.plot(results_df['Date'], results_df['Cumulative_Strategy_Return'], 
         label='DeepAR Strategy', color='green')
plt.plot(results_df['Date'], results_df['Cumulative_Conservative_Return'], 
         label='Conservative DeepAR', color='purple')
plt.title('Cumulative Returns Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 10. Feature Importance Analysis

Let's try to understand which features are most important for our predictions.

In [ ]:
# Create a function to calculate feature importance through permutation
def calculate_permutation_importance(model, dataloader, feature_names, n_repeats=5):
    """Calculate permutation importance for each feature"""
    # Get baseline loss
    baseline_predictions = model.predict(dataloader)
    baseline_loss = model.loss(baseline_predictions.output, baseline_predictions.y)
    baseline_loss = baseline_loss.item()
    
    importances = []
    
    for feature_name in feature_names:
        feature_importances = []
        
        for _ in range(n_repeats):
            # Create a copy of the dataloader with permuted feature
            permuted_dataloader = dataloader.dataset.copy_dataset()
            
            # Permute the feature
            for batch in permuted_dataloader:
                if feature_name in batch.x_cat_names:
                    idx = batch.x_cat_names.index(feature_name)
                    batch.x_cat[:, :, idx] = batch.x_cat[:, :, idx][torch.randperm(batch.x_cat.shape[0])]
                elif feature_name in batch.x_cont_names:
                    idx = batch.x_cont_names.index(feature_name)
                    batch.x_cont[:, :, idx] = batch.x_cont[:, :, idx][torch.randperm(batch.x_cont.shape[0])]
            
            # Get predictions and loss with permuted feature
            permuted_predictions = model.predict(permuted_dataloader)
            permuted_loss = model.loss(permuted_predictions.output, permuted_predictions.y)
            permuted_loss = permuted_loss.item()
            
            # Calculate importance
            feature_importance = permuted_loss - baseline_loss
            feature_importances.append(feature_importance)
        
        # Calculate mean importance across repeats
        importances.append(np.mean(feature_importances))
    
    return pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)

# Try to get feature importance
try:
    feature_importance = calculate_permutation_importance(
        model, val_dataloader, feature_columns, n_repeats=3
    )
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
    plt.title('Top 15 Features by Permutation Importance')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not calculate permutation importance: {e}")

## 11. Conclusion and Next Steps

In this notebook, we've learned how to:

1. Prepare time series data for PyTorch Forecasting
2. Build and train a DeepAR model for forecasting
3. Generate trading signals based on the predictions
4. Evaluate the strategy's performance
5. Create a more conservative version of the strategy

### Potential next steps:

1. **Hyperparameter Optimization**: Use techniques like Optuna to find optimal parameters
2. **Ensemble Methods**: Combine DeepAR with other models for better predictions
3. **Feature Engineering**: Create more advanced features that capture market regimes
4. **Risk Management**: Implement dynamic position sizing based on prediction confidence
5. **Multi-horizon Forecasting**: Predict at multiple time horizons (e.g., 1-day, 3-day, and 5-day ahead)
6. **Walk-forward Validation**: Implement continuous retraining on rolling windows of data